In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [5]:
# Load the training set
df = pd.read_csv(r'.\gamified_dataset/Dyt-desktop.csv')

print("#-----> First 5 rows of the training set:\n")
df.head(5)


#-----> First 5 rows of the training set:



,Gender,Nativelang,Otherlang,Age,Clicks1,Hits1,Misses1,Score1,Accuracy1,Missrate1,...,Score31,Accuracy31,Missrate31,Clicks32,Hits32,Misses32,Score32,Accuracy32,Missrate32,Dyslexia
0,Male,No,Yes,7,10,10,0,10,1.0,0.0,...,0,0.000000,0.00,17,2,0,2,0.117647,0.000000,No
1,Female,Yes,Yes,13,12,12,0,12,1.0,0.0,...,4,0.114286,0.00,26,2,2,2,0.076923,0.076923,Yes
2,Female,No,Yes,7,6,6,0,6,1.0,0.0,...,4,0.114286,0.00,26,1,3,1,0.038462,0.115385,No
3,Female,No,Yes,7,0,0,0,0,0.0,0.0,...,0,0.000000,0.00,1,0,0,0,0.000000,0.000000,No
4,Female,No,Yes,8,4,4,0,4,1.0,0.0,...,1,25.000000,0.05,26,2,2,2,0.076923,0.076923,No


In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import tensorflow as tf

In [7]:
label_encoder = LabelEncoder()
df['Gender'] = label_encoder.fit_transform(df['Gender'])
df['Otherlang'] = label_encoder.fit_transform(df['Otherlang'])
df['Nativelang'] = label_encoder.fit_transform(df['Nativelang'])
df['Dyslexia'] = label_encoder.fit_transform(df['Dyslexia'])

In [8]:
X_features = df.drop(['Dyslexia'], axis=1)
y_labels= df['Dyslexia']

In [10]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels, test_size=0.2, random_state=42, stratify=y_labels)

# Standardize the input features (optional but recommended)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [11]:
from sklearn.metrics import recall_score,classification_report

# Calculate class weights
class_weights = {0: 1, 1: 2}  # Adjust the weight for dyslexia class (1) according to your preference

model = LogisticRegression(class_weight=class_weights)

# Train the model
model.fit(X_train, y_train)

# Cross-validate the model with different decision thresholds
thresholds = np.arange(0.2, 0.3, 0.005)
for threshold in thresholds:
    # Predict probabilities
    y_pred_prob = model.predict_proba(X_test)[:, 1]
    
    # Apply threshold to get binary predictions
    y_pred = (y_pred_prob > threshold).astype(int)
    
    # Evaluate recall on the test set with the current threshold
    recall = recall_score(y_test, y_pred)
    
    # Print the current threshold and recall
    print(f"Threshold: {threshold:.3f}, Recall: {recall:.4f}")
    classification_rep_test = classification_report(y_test, y_pred)
    print(classification_rep_test)


d:\apps n shit\programming\Python\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Threshold: 0.200, Recall: 0.7436
              precision    recall  f1-score   support

           0       0.96      0.82      0.89       651
           1       0.33      0.74      0.46        78

    accuracy                           0.81       729
   macro avg       0.65      0.78      0.67       729
weighted avg       0.90      0.81      0.84       729

Threshold: 0.205, Recall: 0.7436
              precision    recall  f1-score   support

           0       0.96      0.82      0.89       651
           1       0.34      0.74      0.46        78

    accuracy                           0.81       729
   macro avg       0.65      0.78      0.68       729
weighted avg       0.90      0.81      0.84       729

Threshold: 0.210, Recall: 0.7436
              precision    recall  f1-score   support

           0       0.96      0.83      0.89       651
           1       0.34      0.74      0.47        78

    accuracy                           0.82       729
   macro avg       0.65      

In [13]:
import pickle

# Select the final threshold and retrain the model if needed
threshold = 0.265
y_pred_prob = model.predict_proba(X_test)[:, 1]
y_pred = (y_pred_prob > threshold).astype(int)

# Evaluate recall on the test set with the current threshold
recall = recall_score(y_test, y_pred)

# Print the current threshold and recall
print(f"Threshold: {threshold:.3f}, Recall: {recall:.4f}")
classification_rep_test = classification_report(y_test, y_pred)
print(classification_rep_test)

# Save the model to a .sav file
filename = 'lr_model.sav'
with open(filename, 'wb') as file:
    pickle.dump(model, file)

pickle.dump(scaler, open("scaler.sav", "wb"))

Threshold: 0.265, Recall: 0.7051
              precision    recall  f1-score   support

           0       0.96      0.86      0.91       651
           1       0.37      0.71      0.49        78

    accuracy                           0.84       729
   macro avg       0.67      0.78      0.70       729
weighted avg       0.90      0.84      0.86       729

